In [2]:
import numpy as np
import random
from copy import deepcopy

In [3]:
def is_feasible(solution, A, b):
    return np.all(np.dot(A, solution) <= b)

In [4]:
def calc_fitness(solution, c):
    return np.dot(c, solution)    

In [5]:
# Brute-force method for validation
def brute_force_knapsack(A, b, c):
    n = len(c)
    best_fitness = 0
    best_solution = None

    # Iterate over all possible combinations of x
    for x in range(2 ** n):
        solution = [int(i) for i in bin(x)[2:].zfill(n)]

        # Check constraints
        if is_feasible(solution, A, b):
            fitness = calc_fitness(solution, c)
            if fitness > best_fitness:
                best_fitness = fitness
                best_solution = solution

    return best_fitness, best_solution

In [6]:
def generate_problem_instance(m, n, seed=42):
    np.random.seed(seed)
    A = np.random.randint(1, 50, size=(m,n))
    b = np.random.randint(10, 100, size=m)
    c = np.random.randint(1, 100, size=n)
    return A, b, c

In [7]:
def initialize(n, A, b):
    solution = random.choices([1,0], k=n)
    while is_feasible(solution, A, b) == False:
        solution = random.choices([1,0], k=n)
    return solution

In [8]:
def make_small_change(solution, A, b):
    num_changes = max(1, random.randrange(0, max(1, len(solution) // 5)))
    indices = random.sample(range(len(solution)), num_changes)
    for j in indices:
        solution[j] = 1 - solution[j]

    return solution

In [9]:
# Simulated Annealing
def simulated_annealing(A, b, c, initial_solution=None, max_iter=20000, initial_temp=1000, min_temp=1, cooling_rate=0.999):
    n = len(c)

    if initial_solution is None:
        solution = initialize(n, A, b)
    else:
        solution = initial_solution.copy()
    fitness = calc_fitness(solution, c)
    best_solution = solution.copy()
    best_fitness = fitness

    temperature = initial_temp

    for _ in range(max_iter):
        if temperature <= min_temp:
            break
            
        new_solution = solution.copy()
        new_solution = make_small_change(new_solution, A, b)

        if is_feasible(new_solution, A, b):
            new_fitness = calc_fitness(new_solution, c)
            delta = new_fitness - fitness # - best_fitness ?

            if delta > 0 or random.random() < np.exp(delta / temperature):
                solution = new_solution.copy()
                fitness = new_fitness
                if new_fitness > best_fitness:
                    best_solution = new_solution.copy()
                    best_fitness = new_fitness
        
        temperature = max(min_temp, temperature * cooling_rate)
    return best_fitness, best_solution

In [10]:
class Individual:
    def __init__(self, n, A, b, c):
        self.code = initialize(n, A, b)
        self.n = n
        self.A = A
        self.b = b
        self.c = c
        self.calculate_fitness()

    def calculate_fitness(self):
        if is_feasible(self.code, self.A, self.b):
            self.fitness = calc_fitness(self.code, c)
        else:
            self.fitness = float('-inf')

In [11]:
def selection(population, tournament_size):
    k = min(len(population), tournament_size)
    participants = random.sample(population, k)
    return max(participants, key=lambda x: x.fitness)

In [12]:
# randrange > randint
def crossover(parent1, parent2, child1, child2):
    if parent1.n > 2:
        breakpoint = random.randrange(1, parent1.n-1)
        child1.code[:breakpoint] = parent1.code[:breakpoint]
        child1.code[breakpoint:] = parent2.code[breakpoint:]

        child2.code[:breakpoint] = parent2.code[:breakpoint]
        child2.code[breakpoint:] = parent1.code[breakpoint:]
    elif parent1.n == 2:
        child1.code[0] = parent1.code[0]
        child1.code[1] = parent2.code[1]

        child2.code[0] = parent2.code[0]
        child2.code[1] = parent1.code[1]
    else:
        child1.code = parent1.code
        child2.code = parent2.code

In [13]:
def mutation(child, mutation_prob):
    for i in range(child.n):
        if random.random() < mutation_prob:
            child.code[i] = 1 - child.code[i]

In [14]:
# Genetic Algorithm
def genetic_algorithm(A, b, c, population_size=100, tournament_size=40, generations=200, mutation_prob=0.07, elitism_size=2):
    n = len(c)

    population = [Individual(n, A, b, c) for _ in range(population_size)]
    new_population = [Individual(n, A, b, c) for _ in range(population_size)]
    
    for _ in range(generations):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = deepcopy(population[:elitism_size])

        for i in range(elitism_size, population_size, 2):
            parent1 = selection(population, tournament_size)
            tmp, parent1.fitness = parent1.fitness, float('-inf')
            parent2 = selection(population, tournament_size)
            parent1.fitness = tmp

            crossover(parent1, parent2, new_population[i], new_population[i+1])

            mutation(new_population[i], mutation_prob)
            mutation(new_population[i+1], mutation_prob)

            new_population[i].calculate_fitness()
            new_population[i+1].calculate_fitness()

        population = deepcopy(new_population)

    best_individual = max(population, key=lambda x: x.fitness)
    return best_individual.fitness, best_individual.code

In [15]:
# Genetic Algorithm
def ga_plus_sa(A, b, c, population_size=100, tournament_size=40, generations=100, mutation_prob=0.07, elitism_size=2):
    n = len(c)

    population = [Individual(n, A, b, c) for _ in range(population_size)]
    new_population = [Individual(n, A, b, c) for _ in range(population_size)]
    
    for _ in range(generations):
        population.sort(key=lambda x: x.fitness, reverse=True)
        new_population[:elitism_size] = deepcopy(population[:elitism_size])

        for i in range(elitism_size, population_size, 2):
            parent1 = selection(population, tournament_size)
            tmp, parent1.fitness = parent1.fitness, float('-inf')
            parent2 = selection(population, tournament_size)
            parent1.fitness = tmp

            crossover(parent1, parent2, new_population[i], new_population[i+1])

            mutation(new_population[i], mutation_prob)
            mutation(new_population[i+1], mutation_prob)

            new_population[i].calculate_fitness()
            new_population[i+1].calculate_fitness()

            if is_feasible(new_population[i].code, A, b):
                fitness, solution = simulated_annealing(A, b, c, initial_solution=new_population[i].code, max_iter=1000)
                if is_feasible(solution, A, b):
                    new_population[i].code = solution.copy()
                    new_population[i].fitness = fitness
            
            if is_feasible(new_population[i].code, A, b):
                fitness, solution = simulated_annealing(A, b, c, initial_solution=new_population[i+1].code, max_iter=1000)
                if is_feasible(solution, A, b):
                    new_population[i+1].code = solution.copy()
                    new_population[i+1].fitness = fitness

        population = deepcopy(new_population)

    best_individual = max(population, key=lambda x: x.fitness)
    return best_individual.fitness, best_individual.code

In [16]:
m, n = 10, 15
A, b, c = generate_problem_instance(m, n)

print("Matrix A:")
print(A)
print("Vector b:", b)
print("Vector c:", c)

#Solve with brute-force"
print("\nSolving with brute-force:")
bf_fitness, bf_solution = brute_force_knapsack(A, b, c)
print("Optimal value:", bf_fitness)
print("Optimal solution:", bf_solution)

print("\nSolving with simulated_annealing:")
sa_fitness, sa_solution = simulated_annealing(A, b, c)
print("Optimal value:", sa_fitness)
print("Optimal solution:", sa_solution)

print("\nSolving with genetic_algorithm:")
ga_fitness, ga_solution = genetic_algorithm(A, b, c)
print("Optimal value:", ga_fitness)
print("Optimal solution:", ga_solution)

print("\nSolving with ga_sa_hybrid:")
ga_plus_sa_fitness, ga_plus_sa_solution = ga_plus_sa(A, b, c)
print("Optimal value:", ga_plus_sa_fitness)
print("Optimal solution:", ga_plus_sa_solution)

Matrix A:
[[39 29 15 43  8 21 39 19 23 11 11 24 36 40 24]
 [ 3 22  2 24 44 30 38  2 21 33 12 22 44 25 49]
 [27 42 28 16 15 47 44  3 37  7 21  9 39 18  4]
 [25 14  9 26  2 20 28 47  7 44  8 47 35 14 17]
 [36 40  4  2  6 42  4 29 18 26 44 34 10 36 14]
 [31 48 15  8 14 23 40 21 16 45 18 47 24 26 25]
 [45 41 29 15 45  1 25  7  9 24  1 44  8 24 11]
 [17  8 35 35 33  5 42 39 41 28  7  9  8 12 34]
 [33 48 23 24 37 35 44 40 22 27 35  1 35 37 47]
 [14  3  1  5 26 14 39 27  9 15 15 26 42 13 32]]
Vector b: [61 13 32 24 52 38 45 22 41 80]
Vector c: [59 86 28 66 42 45 62 57  6 28 28 44 84 30 62]

Solving with brute-force:
Optimal value: 28
Optimal solution: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

Solving with simulated_annealing:
Optimal value: 28
Optimal solution: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

Solving with genetic_algorithm:
Optimal value: 28
Optimal solution: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]

Solving with ga_sa_hybrid:
Optimal value: 28
Optimal solution: [0, 0

In [17]:
test_cases = [
    (np.array([[1]]), np.array([1]), np.array([10])),
    (np.array([[1, 2]]), np.array([3]), np.array([10, 15])),
    (np.array([[1, 2, 3]]), np.array([5]), np.array([10, 15, 40])),
    (np.array([[2, 3, 1, 4, 5], [3, 2, 2, 5, 4]]), np.array([10, 8]), np.array([20, 30, 40, 10, 50])),
    (np.array([[1, 1, 1, 1, 1, 1]]), np.array([3]), np.array([5, 4, 3, 2, 1, 10])),
    (np.array(np.random.randint(1, 5, size=(3, 12))), np.array([10, 15, 20]), np.array(np.random.randint(1, 100, size=12))),
    (np.array([np.ones(15)]), np.array([20]), np.array(np.random.randint(1, 50, size=15))),
    (np.array([[5, 6, 7, 8, 9, 10, 11, 12],[5, 6, 7, 8, 9, 10, 11, 12]]), np.array([3, 3]), np.array([10, 20, 30, 40, 50, 60, 70, 80])),
    (np.array([[10, 20, 30, 40, 50]]), np.array([5]), np.array([100, 200, 300, 400, 500])),
    (np.array([[1, 1, 1, 1, 1]]), np.array([5]), np.array([10, 20, 30, 40, 50]))
]

optimal_fitnessess = [10, 25, 55, 120, 19, -200, 99999, 0, 0, 150]
optimal_solutions = [[1], [1,1], [0,1,1], [0,1,1,0,1], [1,1,0,0,0,1], [-200], [1,1,1], [0,0,0,0,0,0,0,0], [0,0,0,0,0], [1,1,1,1,1]]

for i, (A, b, c) in enumerate(test_cases):
    print(f"Test {i+1}:")
    bf_fitness, bf_solution = brute_force_knapsack(A, b, c)
    sa_fitness, sa_solution = simulated_annealing(A, b, c)
    ga_fitness, ga_solution = genetic_algorithm(A, b, c)
    ga_plus_sa_fitness, ga_plus_sa_solution = ga_plus_sa(A, b, c)
    print("Optimal fitness:", bf_fitness)
    print("SA fitness:", sa_fitness)
    print("GA fitness:", ga_fitness)
    print("GA_SA fitness:", ga_plus_sa_fitness)
    print("And it should be:", optimal_fitnessess[i])
    print("Solution:      ", bf_solution)
    print("SA solution:   ", sa_solution)
    print("GA solution:   ", ga_solution)
    print("GA_SA solution:", ga_plus_sa_solution)
    print("And it should be:", optimal_solutions[i])
    print("-" * 30)

Test 1:
Optimal fitness: 10
SA fitness: 10
GA fitness: 10
GA_SA fitness: 10
And it should be: 10
Solution:       [1]
SA solution:    [1]
GA solution:    [1]
GA_SA solution: [1]
And it should be: [1]
------------------------------
Test 2:
Optimal fitness: 25
SA fitness: 25
GA fitness: 25
GA_SA fitness: 25
And it should be: 25
Solution:       [1, 1]
SA solution:    [1, 1]
GA solution:    [1, 1]
GA_SA solution: [1, 1]
And it should be: [1, 1]
------------------------------
Test 3:
Optimal fitness: 55
SA fitness: 55
GA fitness: 55
GA_SA fitness: 55
And it should be: 55
Solution:       [0, 1, 1]
SA solution:    [0, 1, 1]
GA solution:    [0, 1, 1]
GA_SA solution: [0, 1, 1]
And it should be: [0, 1, 1]
------------------------------
Test 4:
Optimal fitness: 120
SA fitness: 120
GA fitness: 120
GA_SA fitness: 120
And it should be: 120
Solution:       [0, 1, 1, 0, 1]
SA solution:    [0, 1, 1, 0, 1]
GA solution:    [0, 1, 1, 0, 1]
GA_SA solution: [0, 1, 1, 0, 1]
And it should be: [0, 1, 1, 0, 1]
-